# Sea level budget

In [1]:
import sys
sys.path.append('../code')
import importlib
from time import gmtime, strftime

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
import xarray as xr

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

import SeaLevelContrib as slc

In [2]:
importlib.reload(slc)

<module 'SeaLevelContrib' from '/Users/dewilebars/Projects/SLBudget/notebooks/../code/SeaLevelContrib.py'>

In [3]:
path_outputs = '../outputs/'

## Namelist

In [4]:
# Pick a region name (see choices bellow): 
location_name = 'ZSM' # Name of tide gauge selection or region, 
                      # see options in the next cell
opt_sl = 'tide_gauge' # Use sea level observations from 'tide_gauge' or from 
                      #satellite altimetry ('altimetry')
opt_steric = ['EN4_22',['EBB','NWS'],[500, 4000], [1, 13]]
                # [0]: Poduct: 'EN4_21', 'EN4_22' and 'IAP' available so far
                # [1]: Region on which to average the steric effect: 
                # ENS: Extended North Sea, 
                # EBB: Extended Bay of Biscay
                # BB: Bay of Biscay
                # NWS: Norwegian Sea
                # [2]: Depth of integration in meters
                # [3]: Number of years to use in smoothing window
                # Option 1, 2, 3 can be lists for a 2 layers steric representation
                # Example 1 layer: ['EN4_22','NWS',3000, 9]
                # Example 2 layers: ['EN4_22',['EBB','NWS'],[500, 4000], [1, 13]]
opt_glaciers = 'fred20' # fred20
                            # marzeion15 for Marzeion et al. 2015
                            # zemp19 for Zemp et al. 2019
opt_antarctica = 'Gravis' #'fred20' # rignot19, imbie18, fred20
opt_greenland = 'Gravis'  #'fred20' # mouginot19, fred20
                             # IMBIE is used in Greenland.ipynb but not yet implemented here
opt_tws = 'fred20' # humphrey19 for Humphrey and Gudmundson 2019
                       # fred20 for Frederikse et al. 2020
#!!! Nodal cycle only included for regression method, not yet for dynamical model
opt_wind_ibe = ['regression','ERA5'] # Method 'regression' or 'dynamical_model' 
                          # 'NCEP1': Regression to NCEP1 (1948 to now)
                          # 'ERA5' : Regression to ERA5 (1979 to now)
                          # 'WAQUA' : Use the barotropic ocean model WAQUA
                          # 'GTSM': Use the barotropic ocean model GTSM
opt_nodal = 'potential' # 'potential' or 'regression', 'regression' only available 
                         #if opt_wind_ibe[0] is also 'regression'
global_steric = 'fred20' # None for no splitting between global and local
                     # 'levitus' or 'fred20' otherwise


In [5]:
reg_dict = {'ZSM' : [20, 22, 23, 24, 25, 32], # Zeespiegelmonitor (Dutch coast)
            'All_NL' : [20, 22, 23, 24, 25, 32, 236, 1551], # All Dutch tide gauges from PSMSL
            # North Sea Frederikse et al. 2016
            'NS_Fred16': [20, 32, 236, 7, 80, 89, 302, 47, 830, 361, 95, 754],
            'NS_waqua': [20, 32, 236, 7, 80, 47, 361, 95, 754],
            'Newlyn': [202], 
            'Brest': [1], 
            'Vlissingen': [20],
            'Delfzijl': [24],
            
            # Also define regions with lat/lon polygones.
            # Those can only bse used for opt_sl = 'altimetry'
            'NL_region' : np.array([[2.5, 53], [3.3, 51.5], [4.25, 52.25], 
                                    [4.75, 53.3], [5.5, 53.6], [7, 53.75], 
                                    [7, 55], [4, 54.5]])
           }

location = reg_dict[location_name]

if type(location) == list:
    print('Using a list of tide gauge IDs')
elif type(location) == np.ndarray:
    print('Using a region defined by a polygone')
    coord = [np.mean(location[:,1]), np.mean(location[:,0])]
    print('Using region center [lat/lon] for fingerprints:')
    print(coord)

print('All location:')
print(location)

Using a list of tide gauge IDs
All location:
[20, 22, 23, 24, 25, 32]


## Plot the region of interest or tide gauges location

## Budget

In [6]:
# Check the tide gauge data
#tg_df = slc.tide_gauge_obs(location, interp=True)
#tg_df

In [7]:
slmean_df = slc.local_budget(location, opt_sl, opt_steric, opt_glaciers, 
                             opt_antarctica, opt_greenland, opt_tws, opt_wind_ibe, 
                             opt_nodal, global_steric, True)


20
22
23
24
25
32
Working on tide gauge id: 20
with lat/lon (51.442222, 3.596111)
found point 51.5 3.5
Working on tide gauge id: 22
with lat/lon (51.9775, 4.12)
found point 52.0 4.0
Working on tide gauge id: 23
with lat/lon (52.964444, 4.745)
found point 53.0 4.75
Working on tide gauge id: 24
with lat/lon (53.326389, 6.933056)
found point 53.25 7.0
Working on tide gauge id: 25
with lat/lon (53.175556, 5.409444)
found point 53.25 5.5
Working on tide gauge id: 32
with lat/lon (52.462222, 4.554722)
found point 52.5 4.5


In [8]:
#slmean_df = slmean_df.loc[1958:2014] # Select date to fit Fred16 study

#slmean_df = slmean_df.loc[1993:]
slmean_df = slmean_df.loc[1960:]

__Note:__
The method to restrict the period of interest above still uses the regression over the full maximum available time. Usually, restricting the period in the code (in function tide_gauge_obs) results in slightly better results.

In [9]:
display(slmean_df.iloc[:5])
display(slmean_df.iloc[-5:])

,Total,LocSteric,GloSteric,GIA,Glaciers,Antarctica,Greenland,TWS,Nodal,Wind,Pressure,Obs
time,,,,,,,,,,,,
1960,1.326524,-1.541605,1.172172,0.736667,1.192500,0.095833,0.115833,-0.508333,-0.052226,-2.163200,2.278883,1.007686
1961,5.239173,-1.491514,1.359791,0.773500,1.235000,0.101667,0.124167,-0.360000,-0.016606,2.858693,0.654476,7.774352
1962,3.528583,-2.773398,1.488711,0.810333,1.287500,0.105833,0.133333,-0.424167,0.076486,2.736963,0.086987,2.524352
1963,-2.557330,-3.987076,1.355248,0.847167,1.316667,0.111667,0.135000,-0.519167,0.216538,-2.614893,0.581520,-3.208981
1964,-1.559614,-1.368957,1.121032,0.884000,1.312500,0.115833,0.135833,-0.673333,0.387736,-2.340016,-1.134244,-1.158981


,Total,LocSteric,GloSteric,GIA,Glaciers,Antarctica,Greenland,TWS,Nodal,Wind,Pressure,Obs
time,,,,,,,,,,,,
2020,19.169670,3.240667,5.562259,2.946667,3.432379,1.076499,0.363772,-0.982955,0.418580,2.394166,0.717636,17.957686
2021,15.518267,2.557163,5.688242,2.983500,3.481091,1.105715,0.368343,-0.981076,0.601550,-0.252124,-0.034138,15.841019
2022,17.893691,3.548858,5.814225,3.020333,3.529803,1.052757,0.372563,-0.979197,0.772196,1.311067,-0.548913,17.124352
2023,23.141346,5.204189,5.940207,3.057167,3.578515,1.042485,0.378161,-0.977318,0.911247,2.711520,1.295173,22.941019
2024,21.516160,5.881344,6.066190,3.094000,3.627227,1.084876,0.382869,-0.975439,1.003005,0.218122,1.133965,23.574352


In [10]:
fig, ax = slc.plot_budget(location_name, slmean_df)

In [15]:
fig2, ax2 = slc.plot_budget_ts(slmean_df)

<Axes: xlabel='time'>

In [16]:
diff = slmean_df['Obs']-slmean_df['Total']
diff = diff-diff.mean()
diff.plot()

<Axes: xlabel='time'>

In [17]:
diff_ra3 = diff.rolling(3,center=True).mean()
diff_ra3.plot()

In [18]:
budget_name = (f'slbudget_{strftime("%Y_%m_%d", gmtime())}_{location_name}_{opt_sl}_'+
               f'{slmean_df.index[0]}_{slmean_df.index[-1]}_{opt_wind_ibe[1]}_'+
               f'Steric{opt_steric[0]}_{opt_steric[2]}m')

# Write out a budget csv file
slmean_df.to_csv(path_outputs+budget_name+'.csv')

# Write out the figure
fig.savefig(path_outputs+budget_name+'.pdf', bbox_inches='tight') #pad_inches=2

# Write out second figure
fig2.savefig(path_outputs + budget_name + '_budgetplot.pdf', bbox_inches='tight')

# Write budget info to a text file
text_out = ('Budget information: \n'
            'Tide gauge selection: '+location_name+' \n'
            'Sea level data used: '+opt_sl+' \n'
            'Corresponding to ids: '+str(location)+' \n'
            'Period: '+str(slmean_df.index[0])+'-'+str(slmean_df.index[-1])+' \n'
            'Steric contribution: '+str(opt_steric)+' \n'
            'Global steric: '+global_steric+' \n'
            'Glaciers: '+opt_glaciers+' \n'
            'Antarctica:'+opt_antarctica+' \n'
            'Greenland: '+opt_greenland+' \n'
            'Terrestrial water storage: '+opt_tws+' \n'
            'Wind and inverse barometer effect: '+str(opt_wind_ibe)+' \n'
           )

file = open(path_outputs+budget_name+'.txt','w') 
file.write(text_out)
file.close()

# Alternative for better tracability but less lisibility:
# Generate a unique number that describes the budget, number the options and give an order:21324

### Some remarks

Comment on the choice of steric expansion location:
It seems that using the Bay of Biscay reduces the eror compared to extended North Sea.
On the other hand Norwegian Sea increases it.

Computing the wind, nodal and pressure effects (linear fit) after removing the trend from the reconstruction improves the fit compared to computing the fit using a linear trend.

Using the extended Bay of Biscay area, mostly solves the trend problem but it introduces higher rms (0.27) and ae (1.30).

The same conclusion applies to stopping the integration of steric effect at 60N instead of the standard 69N. This is because steric trend in the region 60-69 is smaller than further South (see the Steric notebook). What is the most relevent for the North Sea?

Using sea level from WAQUA forced by ERA-interim increases the rmse (0.26) and ae (1.35) compared to linear regressions to ERA5 or NCEP1 reanalysis. This might be because of sea level variations external to the domain entering the domain.

One possible difference between KNMI projections and Deltares zsm is the vertical land motion: GIA from ICE6G provides 0.36mm/y for the Dutch tide gauges while 0.45 was used in the ZSM. That is still not much: 2 mm in 20 years.

The computation of acceleration is very sensitive to the period used to compute it even for the steric effect. Also it seems very difficult to get right. For example at the Brest tide gauge, observations show a deceleration while the budget shows an acceleration.
This points toward the need to consider longer time periods to study the acceleration.

Even in the period 1950-2017 the acceleration is difficult to get right. For the Dutch coast the budget gives twice larger values than the tide gauge observations. For that period it seems that for Newlyn, Brest and Dutch coast the trend is underestimated and the acceleration is overestimated.

Acceleration for the Dutch coast is very sensitive to the wind and steric effects: check starting point in 1950 compared to 1958 -> Also check over the ERA5 period

__12/2021__

Between 1950 and 1960 IAP is a lot better than EN4. EN4 has a strange behaviour there that is not backed up by tide gauge data.
Integrating steric sea level up to 1000m improves North Sea compared to 2000m. But not the Dutch coast. Probably because of underestimatd GIA?

Unfortunately the choice of region and product used for the steric sea level matters.

Large error using EBB in 1961 when sea level rose but the budget doesn't see it.